In [1]:
import os
import sys

import os
import pickle

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
color = sns.color_palette()

import unsupervised_learning_util as utl
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.random_projection import GaussianRandomProjection
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, FastICA
from sklearn.manifold import TSNE

plt.tight_layout()
plt.style.use("ggplot")
mpl.rcParams['figure.figsize'] = [8, 6]
mpl.rcParams['figure.dpi'] = 200
mpl.rcParams['savefig.dpi'] = 500


NJOBS = 32
VERBOSE = 0
limit = 5000

folder = "DimensionalityReduction/"
utl.check_folder(folder)

%matplotlib inline

C:\Users\joshu\OneDrive - Georgia Institute of Technology\Georgia-Tech\CS 7641 - Machine Learning\Assignments\Unsupervised Learning and Dimensionality Reduction\DimensionalityReduction/ folder already exists.


# Load Data

In [2]:
gathered_data = utl.setup(["MNIST"])
gathered_data_fashion = utl.setup(["Fashion-MNIST"])

mnist = {}
fashion_mnist = {}
mnist_not_scaled = {}
fashion_mnist_not_scaled = {}

mnist['train_X'], mnist['train_y'], \
mnist['valid_X'], mnist['valid_y'], \
mnist['test_X'], mnist['test_y'] = utl.split_data(gathered_data["MNIST"]["X"],
                                                  gathered_data["MNIST"]["y"], minMax=True)
mnist_not_scaled['train_X'], mnist_not_scaled['train_y'], \
mnist_not_scaled['valid_X'], mnist_not_scaled['valid_y'], \
mnist_not_scaled['test_X'], mnist_not_scaled['test_y'] = utl.split_data(
    gathered_data["MNIST"]["X"], gathered_data["MNIST"]["y"], scale=False)

fashion_mnist['train_X'], fashion_mnist['train_y'], \
fashion_mnist['valid_X'], fashion_mnist['valid_y'], \
fashion_mnist['test_X'], fashion_mnist['test_y'] = utl.split_data(gathered_data_fashion["Fashion-MNIST"]["X"],
                                                                  gathered_data_fashion["Fashion-MNIST"]["y"],
                                                                  minMax=True)

fashion_mnist_not_scaled['train_X'], fashion_mnist_not_scaled['train_y'], \
fashion_mnist_not_scaled['valid_X'], fashion_mnist_not_scaled['valid_y'], \
fashion_mnist_not_scaled['test_X'], fashion_mnist_not_scaled['test_y'] = utl.split_data(
    gathered_data_fashion["Fashion-MNIST"]["X"], gathered_data_fashion["Fashion-MNIST"]["y"], scale=False)

dataset folder already exists.
MNIST dataset found:
	Loading MNIST.feather
	Finished loading MNIST dataset
dataset folder already exists.
Fashion-MNIST dataset found:
	Loading Fashion-MNIST.feather
	Finished loading Fashion-MNIST dataset


# Only Run if you need to recreate the reduced datasets without going through the optimization processes

In [3]:
def combine_and_save(X, y, dataset_name, algorithm_name):
    X.reset_index(inplace=True, drop=True)
    X["Label"] = y
    with open(f"{os.getcwd()}/DimensionalityReduction/{algorithm_name}_{dataset_name}_Reduced_Dataset.pkl", "wb") as output_file:
        pickle.dump(X, output_file)
        output_file.close()

In [4]:
limit=10000
# PCA
pca_mnist = PCA(n_components=326).fit_transform(mnist['train_X'].iloc[:limit, :])
pca_fashion = PCA(n_components=445).fit_transform(fashion_mnist['train_X'].iloc[:limit, :])

In [5]:
combine_and_save(X=pd.DataFrame(pca_mnist), y=mnist['train_y'].iloc[:limit], dataset_name="MNIST", algorithm_name="PCA")
combine_and_save(X=pd.DataFrame(pca_fashion), y=fashion_mnist['train_y'].iloc[:limit], dataset_name="Fashion", algorithm_name="PCA")

In [6]:
# ICA
ica_mnist = FastICA(n_components=284, whiten=True, max_iter=400).fit_transform(mnist['train_X'].iloc[:limit, :])
ica_fashion = FastICA(n_components=412, whiten=True, max_iter=400).fit_transform(fashion_mnist['train_X'].iloc[:limit, :])

C:\ProgramData\Anaconda3\envs\CS7641\lib\site-packages\sklearn\decomposition\_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


In [7]:
combine_and_save(X=pd.DataFrame(ica_mnist), y=mnist['train_y'].iloc[:limit], dataset_name="MNIST", algorithm_name="ICA")
combine_and_save(X=pd.DataFrame(ica_fashion), y=fashion_mnist['train_y'].iloc[:limit], dataset_name="Fashion", algorithm_name="ICA")

In [8]:
# RP
rp_mnist = GaussianRandomProjection(n_components=715).fit_transform(mnist['train_X'].iloc[:limit, :])
rp_fashion = GaussianRandomProjection(n_components=746).fit_transform(fashion_mnist['train_X'].iloc[:limit, :])

In [9]:
combine_and_save(X=pd.DataFrame(rp_mnist), y=mnist['train_y'].iloc[:limit], dataset_name="MNIST", algorithm_name="RP")
combine_and_save(X=pd.DataFrame(rp_fashion), y=fashion_mnist['train_y'].iloc[:limit], dataset_name="Fashion", algorithm_name="RP")

In [10]:
# RF
rf_mnist = RandomForestClassifier(n_estimators=1000, n_jobs=-1, verbose=10, warm_start=True)
X = mnist['train_X'].iloc[:limit, :]
y = mnist['train_y'].iloc[:limit]
rf_mnist.fit(X, y)
rf_mnist_df = pd.DataFrame(columns=["Feature_Importance"], data=rf_mnist.feature_importances_)
rf_mnist_df.sort_values(by=["Feature_Importance"], ascending=False, inplace=True)
rf_mnist_feature_index = rf_mnist_df.copy().index
mnist_idx = rf_mnist_feature_index.to_numpy()
mnist_idx = mnist_idx[:152]
mnist_df = X.iloc[:limit, mnist_idx]
result_X = pd.DataFrame(mnist_df)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 36 concurrent workers.


building tree 1 of 1000building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000

building tree 5 of 1000
building tree 6 of 1000building tree 7 of 1000

building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000building tree 19 of 1000

building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000building tree 25 of 1000

building tree 26 of 1000building tree 27 of 1000
building tree 28 of 1000building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000

building tree 32 of 1000

building tree 33 of 1000
building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building 

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    0.4s



building tree 81 of 1000building tree 82 of 1000building tree 83 of 1000


building tree 84 of 1000building tree 85 of 1000building tree 86 of 1000building tree 87 of 1000
building tree 88 of 1000

building tree 89 of 1000building tree 90 of 1000building tree 91 of 1000


building tree 92 of 1000building tree 93 of 1000
building tree 94 of 1000building tree 95 of 1000building tree 96 of 1000

building tree 97 of 1000building tree 98 of 1000



building tree 99 of 1000

building tree 100 of 1000
building tree 101 of 1000building tree 102 of 1000
building tree 103 of 1000
building tree 104 of 1000building tree 105 of 1000


building tree 106 of 1000building tree 107 of 1000


building tree 108 of 1000
building tree 109 of 1000
building tree 110 of 1000
building tree 111 of 1000
building tree 112 of 1000
building tree 113 of 1000
building tree 114 of 1000building tree 115 of 1000

building tree 116 of 1000
building tree 117 of 1000
building tree 118 of 1000
building tree 119 of 1000build

[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    0.6s


...

[Parallel(n_jobs=-1)]: Done 810 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 853 tasks      | elapsed:    2.7s


building tree 916 of 1000
building tree 917 of 1000
building tree 918 of 1000building tree 919 of 1000
building tree 920 of 1000building tree 921 of 1000building tree 922 of 1000

building tree 923 of 1000

building tree 924 of 1000
building tree 925 of 1000
building tree 926 of 1000

building tree 927 of 1000
building tree 928 of 1000building tree 929 of 1000
building tree 930 of 1000building tree 931 of 1000
building tree 932 of 1000
building tree 933 of 1000building tree 934 of 1000building tree 935 of 1000building tree 936 of 1000
building tree 937 of 1000building tree 938 of 1000
building tree 939 of 1000





building tree 940 of 1000building tree 941 of 1000
building tree 942 of 1000building tree 943 of 1000building tree 944 of 1000



building tree 945 of 1000building tree 946 of 1000building tree 947 of 1000

building tree 948 of 1000
building tree 949 of 1000


building tree 950 of 1000
building tree 951 of 1000
building tree 952 of 1000
building tree 953 of 1000
building tre

[Parallel(n_jobs=-1)]: Done 896 tasks      | elapsed:    2.8s


building tree 988 of 1000
building tree 989 of 1000
building tree 990 of 1000building tree 991 of 1000building tree 992 of 1000


building tree 993 of 1000building tree 994 of 1000building tree 995 of 1000building tree 996 of 1000building tree 997 of 1000
building tree 998 of 1000
building tree 999 of 1000

building tree 1000 of 1000





[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.1s finished


In [11]:
combine_and_save(X=result_X, y=y, dataset_name="MNIST", algorithm_name="RF")

In [12]:
rf_fashion = RandomForestClassifier(n_estimators=1000, n_jobs=-1, verbose=10, warm_start=True)
X = fashion_mnist['train_X'].iloc[:limit, :]
y = fashion_mnist['train_y'].iloc[:limit]
rf_fashion.fit(X, y)
rf_fashion_df = pd.DataFrame(columns=["Feature_Importance"], data=rf_fashion.feature_importances_)
rf_fashion_df.sort_values(by=["Feature_Importance"], ascending=False, inplace=True)
rf_fashion_feature_index = rf_fashion_df.copy().index
fashion_idx = rf_fashion_feature_index.to_numpy()
fashion_idx = fashion_idx[:250]
fashion_df = X.iloc[:limit, fashion_idx]
result_X = pd.DataFrame(fashion_df)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 36 concurrent workers.


building tree 1 of 1000
building tree 2 of 1000building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000

building tree 8 of 1000
building tree 9 of 1000building tree 10 of 1000
building tree 11 of 1000

building tree 12 of 1000building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000

building tree 16 of 1000
building tree 17 of 1000building tree 18 of 1000building tree 19 of 1000


building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000

building tree 29 of 1000building tree 30 of 1000

building tree 31 of 1000building tree 32 of 1000
building tree 33 of 1000building tree 34 of 1000building tree 35 of 1000


building tree 36 of 1000

building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building 

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.5s


building tree 80 of 1000

building tree 81 of 1000building tree 82 of 1000

building tree 83 of 1000building tree 84 of 1000

building tree 85 of 1000
building tree 86 of 1000building tree 87 of 1000building tree 88 of 1000


building tree 89 of 1000
building tree 90 of 1000
building tree 91 of 1000building tree 92 of 1000building tree 93 of 1000

building tree 94 of 1000
building tree 95 of 1000
building tree 96 of 1000
building tree 97 of 1000
building tree 98 of 1000building tree 99 of 1000
building tree 100 of 1000building tree 101 of 1000building tree 102 of 1000




building tree 103 of 1000building tree 104 of 1000

building tree 105 of 1000
building tree 106 of 1000building tree 107 of 1000building tree 108 of 1000


building tree 109 of 1000
building tree 110 of 1000
building tree 111 of 1000
building tree 112 of 1000
building tree 113 of 1000building tree 114 of 1000
building tree 115 of 1000building tree 116 of 1000
building tree 117 of 1000building tree 118 of 1000building 

[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:    0.8s


...

[Parallel(n_jobs=-1)]: Done 853 tasks      | elapsed:    4.3s


building tree 927 of 1000
building tree 928 of 1000
building tree 929 of 1000
building tree 930 of 1000building tree 931 of 1000

building tree 932 of 1000
building tree 933 of 1000building tree 934 of 1000
building tree 935 of 1000
building tree 936 of 1000

building tree 937 of 1000building tree 938 of 1000building tree 939 of 1000building tree 940 of 1000



building tree 941 of 1000building tree 942 of 1000

building tree 943 of 1000
building tree 944 of 1000building tree 945 of 1000
building tree 946 of 1000building tree 947 of 1000

building tree 948 of 1000
building tree 949 of 1000
building tree 950 of 1000building tree 951 of 1000


building tree 952 of 1000building tree 953 of 1000

building tree 954 of 1000
building tree 955 of 1000
building tree 956 of 1000
building tree 957 of 1000
building tree 958 of 1000
building tree 959 of 1000
building tree 960 of 1000
building tree 961 of 1000
building tree 962 of 1000
building tree 963 of 1000
building tree 964 of 1000
building tre

[Parallel(n_jobs=-1)]: Done 896 tasks      | elapsed:    4.5s



building tree 966 of 1000
building tree 967 of 1000
building tree 968 of 1000
building tree 969 of 1000building tree 970 of 1000

building tree 971 of 1000building tree 972 of 1000

building tree 973 of 1000building tree 974 of 1000
building tree 975 of 1000building tree 976 of 1000
building tree 977 of 1000
building tree 978 of 1000building tree 979 of 1000



building tree 980 of 1000building tree 981 of 1000building tree 982 of 1000
building tree 983 of 1000

building tree 984 of 1000building tree 985 of 1000
building tree 986 of 1000building tree 987 of 1000

building tree 988 of 1000

building tree 989 of 1000
building tree 990 of 1000
building tree 991 of 1000

building tree 992 of 1000
building tree 993 of 1000
building tree 994 of 1000
building tree 995 of 1000
building tree 996 of 1000
building tree 997 of 1000
building tree 998 of 1000
building tree 999 of 1000
building tree 1000 of 1000


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.9s finished


In [13]:
combine_and_save(X=result_X, y=y, dataset_name="Fashion", algorithm_name="RF")

# Load Reduced Datasets

In [ ]:
with open(f"{os.getcwd()}/DimensionalityReduction/PCA_MNIST_Reduced_Dataset.pkl", "rb") as input_file:
    pca_mnist_reduced_data = pickle.load(input_file)
    input_file.close()
with open(f"{os.getcwd()}/DimensionalityReduction/PCA_Fashion_Reduced_Dataset.pkl", "rb") as input_file:
    pca_fashion_reduced_data = pickle.load(input_file)
    input_file.close()
print(f"PCA MNIST: {pca_mnist_reduced_data.shape}")
print(f"PCA Fashion: {pca_fashion_reduced_data.shape}")

In [ ]:
with open(f"{os.getcwd()}/DimensionalityReduction/ICA_MNIST_Reduced_Dataset.pkl", "rb") as input_file:
    ica_mnist_reduced_data = pickle.load(input_file)
    input_file.close()
with open(f"{os.getcwd()}/DimensionalityReduction/ICA_Fashion_Reduced_Dataset.pkl", "rb") as input_file:
    ica_fashion_reduced_data = pickle.load(input_file)
    input_file.close()
print(f"ICA MNIST: {ica_mnist_reduced_data.shape}")
print(f"ICA Fashion: {ica_fashion_reduced_data.shape}")

In [ ]:
with open(f"{os.getcwd()}/DimensionalityReduction/RF_MNIST_Reduced_Dataset.pkl", "rb") as input_file:
    rf_mnist_reduced_data = pickle.load(input_file)
    input_file.close()
with open(f"{os.getcwd()}/DimensionalityReduction/RF_Fashion_Reduced_Dataset.pkl", "rb") as input_file:
    rf_fashion_reduced_data = pickle.load(input_file)
    input_file.close()
print(f"RF MNIST: {rf_mnist_reduced_data.shape}")
print(f"RF Fashion: {rf_fashion_reduced_data.shape}")

In [ ]:
with open(f"{os.getcwd()}/DimensionalityReduction/RP_MNIST_Reduced_Dataset.pkl", "rb") as input_file:
    rp_mnist_reduced_data = pickle.load(input_file)
    input_file.close()
with open(f"{os.getcwd()}/DimensionalityReduction/RP_Fashion_Reduced_Dataset.pkl", "rb") as input_file:
    rp_fashion_reduced_data = pickle.load(input_file)
    input_file.close()
print(f"RP MNIST: {rp_mnist_reduced_data.shape}")
print(f"RP Fashion: {rp_fashion_reduced_data.shape}")

# K-Means Clustering

In [ ]:
temp_folder = "Clustering/" + "Reduced/"
utl.check_folder(temp_folder)
save_dir = os.getcwd() + temp_folder
limit = 5000
idx = [i for i in range(2, 31, 1)]
cols = ["Inertia", "Silhouette", "Homogeneity", "Completeness", "Harmonic_Mean", "Calinski_Harabasz", "Davies_Bouldin"]

In [ ]:
mnist_results = pd.DataFrame(columns=cols, index=idx,
                               data=np.zeros(shape=(len(idx), len(cols))))

fashion_results = pd.DataFrame(columns=cols, index=idx,
                                          data=np.zeros(shape=(len(idx), len(cols))))
algorithm_names = ["PCA", "RP", "RF", "ICA"]
print("Starting K-Means Clustering")
for _df in ["MNIST", "Fashion-MNIST"]:
    for alg in algorithm_names:
        for k in idx:
            if _df == "MNIST":
                with open(f"{os.getcwd()}/DimensionalityReduction/{alg}_{_df}_Reduced_Dataset.pkl", "rb") as input_file:
                    temp_reduced = pickle.load(input_file)
                    input_file.close()
                data = utl.extract_data_and_labels(temp_reduced)
                temp_train_X = data["train_X"]
                temp_train_y = data["train_y"]
                k_means = KMeans(n_clusters=k, verbose=VERBOSE).fit(temp_train_X)
                inertia = k_means.inertia_
                silhouette_average = silhouette_score(temp_train_X, k_means.labels_, sample_size=limit)
                homogeneity, completeness, v_measure = homogeneity_completeness_v_measure(temp_train_y, k_means.labels_)
                mnist_results.loc[k, "Inertia"] = inertia
                mnist_results.loc[k, "Silhouette"] = silhouette_average
                mnist_results.loc[k, "Calinski_Harabasz"] = calinski_harabasz_score(temp_train_X, k_means.labels_)
                mnist_results.loc[k, "Davies_Bouldin"] = davies_bouldin_score(temp_train_X, k_means.labels_)
                mnist_results.loc[k, "Homogeneity"] = homogeneity
                mnist_results.loc[k, "Completeness"] = completeness
                mnist_results.loc[k, "Harmonic_Mean"] = v_measure
                print(f"\n\t{_df} - k={k} \n{mnist_results.loc[k]}")

            elif _df == "Fashion-MNIST":
                with open(f"{os.getcwd()}/DimensionalityReduction/{alg}_Fashion_Reduced_Dataset.pkl", "rb") as input_file:
                    temp_reduced = pickle.load(input_file)
                    input_file.close()
                data = utl.extract_data_and_labels(temp_reduced)
                temp_train_X = data["train_X"]
                temp_train_y = data["train_y"]
                k_means = KMeans(n_clusters=k, verbose=VERBOSE).fit(temp_train_X)
                inertia = k_means.inertia_
                silhouette_average = silhouette_score(temp_train_X, k_means.labels_, sample_size=limit)
                homogeneity, completeness, v_measure = homogeneity_completeness_v_measure(temp_train_y, k_means.labels_)
                fashion_results.loc[k, "Inertia"] = inertia
                fashion_results.loc[k, "Silhouette"] = silhouette_average
                fashion_results.loc[k, "Calinski_Harabasz"] = calinski_harabasz_score(temp_train_X, k_means.labels_)
                fashion_results.loc[k, "Davies_Bouldin"] = davies_bouldin_score(temp_train_X, k_means.labels_)
                fashion_results.loc[k, "Homogeneity"] = homogeneity
                fashion_results.loc[k, "Completeness"] = completeness
                fashion_results.loc[k, "Harmonic_Mean"] = v_measure
                print(f"\n\t{_df} - k={k} \n{fashion_results.loc[k]}")


In [ ]:
with open(f"{os.getcwd()}/Clustering/Clustered_Reduced_MNIST.pkl", "wb") as output_file:
    pickle.dump(mnist_results, output_file)
    output_file.close()
with open(f"{os.getcwd()}/Clustering/Clustered_Reduced_Fashion.pkl", "wb") as output_file:
    pickle.dump(fashion_results, output_file)
    output_file.close()

In [ ]:
with open(f"{os.getcwd()}/{temp_folder}/MNIST_Results.pkl", "rb") as in_file:
    mnist_results = pickle.load(in_file)
    in_file.close()

In [ ]:
with open(f"{os.getcwd()}/{temp_folder}/Fashion_Results.pkl", "rb") as in_file:
    fashion_results = pickle.load(in_file)
    in_file.close()

In [ ]:
with open(f"{os.getcwd()}/{temp_folder}/MNIST_Results.pkl", "wb") as out_file:
    pickle.dump(mnist_results, out_file)
    out_file.close()

In [ ]:
with open(f"{os.getcwd()}/{temp_folder}/Fashion_Results.pkl", "wb") as out_file:
    pickle.dump(fashion_results, out_file)
    out_file.close()

## Elbow Method

In [ ]:
limit=5000

In [ ]:
plt.close("all")
end = 31
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
# ax1_secondary = ax1.twinx()
# ax2_secondary = ax2.twinx()

mnist_model = KMeans()
mnist_visualizer = KElbowVisualizer(mnist_model, k=(2, end), ax=ax1, timings=False)
mnist_visualizer.fit(mnist["train_X"].iloc[:limit, :])

fashion_model = KMeans()
fashion_visualizer = KElbowVisualizer(fashion_model, k=(2, end), ax=ax2, timings=False)
fashion_visualizer.fit(fashion_mnist["train_X"].iloc[:limit, :])

# mnist_results[["Silhouette", "Homogeneity", "Completeness", "Harmonic_Mean"]].iloc[:end].plot(ax=ax1_secondary, linestyle="--")
ax1.set_title(f"K Means Clustering\nDistortion MNIST", fontsize=15, weight='bold')
# ax1.grid(which='major', linestyle='-', linewidth='0.5', color='white')
ax1.set_xlabel("K Clusters", fontsize=15, weight='heavy')
ax1.set_ylabel("Distortion", fontsize=15, weight='heavy')
ax1.legend(loc="best", markerscale=1.1, frameon=True,
                   edgecolor="black", fancybox=True, shadow=True)

# fashion_results[["Silhouette", "Homogeneity", "Completeness", "Harmonic_Mean"]].iloc[:end].plot(ax=ax2_secondary, linestyle="--")
ax2.set_title(f"K Means Clustering\nDistortion Fashion MNIST", fontsize=15, weight='bold')
# ax2.grid(which='major', linestyle='-', linewidth='0.5', color='white')
ax2.set_xlabel("K Clusters", fontsize=15, weight='heavy')
ax2.set_ylabel("Distortion", fontsize=15, weight='heavy')
ax2.legend(loc="best", markerscale=1.1, frameon=True,
                   edgecolor="black", fancybox=True, shadow=True)

plt.savefig(f"{os.getcwd()}/{temp_folder}KMEans_Elbow_Method_Combined.png", bbox_inches='tight')

## Silhouette Method

In [ ]:
plt.close("all")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
mnist_cluster_count = 10
fashion_cluster_count = 9

mnist_model = KMeans(n_clusters=mnist_cluster_count, random_state=42)
fashion_model = KMeans(n_clusters=fashion_cluster_count, random_state=42)

mnist_vis = SilhouetteVisualizer(mnist_model, ax=ax1, 
                                 colors='yellowbrick').fit(mnist["train_X"].iloc[:limit, :]).finalize()
fashion_mnist_vis = SilhouetteVisualizer(fashion_model, ax=ax2, 
                                         colors='yellowbrick').fit(fashion_mnist["train_X"].iloc[:limit, :]).finalize()

ax1.set_title(f"Silhouette Plot of KMEans Clustering\non MNIST with {mnist_cluster_count} Clusters", fontsize=15, weight='bold')
ax1.legend(loc="best", markerscale=1.1, frameon=True,
                   edgecolor="black", fancybox=True, shadow=True)
ax1.set_xlabel("Silhouette Coefficient Values", fontsize=15, weight='heavy')
ax1.set_ylabel("Cluster Label", fontsize=15, weight='heavy')

ax2.set_title(f"Silhouette Plot of KMEans Clustering\non Fashion MNIST with {fashion_cluster_count} Clusters", fontsize=15, weight='bold')
ax2.legend(loc="best", markerscale=1.1, frameon=True,
                   edgecolor="black", fancybox=True, shadow=True)
ax2.set_xlabel("Silhouette Coefficient Values", fontsize=15, weight='heavy')
ax2.set_ylabel("Cluster Label", fontsize=15, weight='heavy')

plt.tight_layout()

plt.savefig(f"{os.getcwd()}/{temp_folder}KMEans_Silhouette_Combined.png", bbox_inches='tight')

# Expectation Maximization 

In [ ]:
index = np.arange(1, 10, 1).astype(np.int)
types = ["Full"]
columns = ["AIC_Full", "BIC_Full"]
em_mnist_results = pd.DataFrame(columns=columns, index=index, data=np.zeros(shape=(index.shape[0], len(columns))))
em_fashion_results = pd.DataFrame(columns=columns, index=index, data=np.zeros(shape=(index.shape[0], len(columns))))

In [ ]:
X = mnist["train_X"].iloc[:limit, :]
for idx in index:
    print(f"N_Components: {idx}")
    for _type in types:
        temp_gmm = GaussianMixture(n_components=idx, n_init=10, covariance_type=_type.lower(), warm_start=True, max_iter=500).fit(X)
        em_mnist_results.loc[idx, f"AIC_{_type}"] = temp_gmm.aic(X)
        em_mnist_results.loc[idx, f"BIC_{_type}"] = temp_gmm.bic(X)

In [ ]:
with open(f"{os.getcwd()}/{temp_folder}/MNIST_EM_Results.pkl", "wb") as out_file:
    pickle.dump(em_mnist_results, out_file)
    out_file.close()

In [ ]:
X = fashion_mnist["train_X"].iloc[:limit, :]
for idx in index:
    print(f"N_Components: {idx}")
    for _type in types:
        temp_gmm = GaussianMixture(n_components=idx, n_init=10, covariance_type=_type.lower(), warm_start=True, max_iter=500).fit(X)
        em_fashion_results.loc[idx, f"AIC_{_type}"] = temp_gmm.aic(X)
        em_fashion_results.loc[idx, f"BIC_{_type}"] = temp_gmm.bic(X)


In [ ]:
with open(f"{os.getcwd()}/{temp_folder}/Fashion_EM_Results.pkl", "wb") as out_file:
    pickle.dump(em_fashion_results, out_file)
    out_file.close()

In [ ]:
with open(f"{os.getcwd()}/{temp_folder}/MNIST_EM_Results.pkl", "rb") as in_file:
    em_mnist_results = pickle.load(in_file)
    in_file.close()

In [ ]:
with open(f"{os.getcwd()}/{temp_folder}/Fashion_EM_Results.pkl", "rb") as in_file:
    em_fashion_results = pickle.load(in_file)
    in_file.close()

In [ ]:
plt.close("all")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

em_mnist_results[["AIC_Full", "BIC_Full"]].plot(ax=ax1)
ax1.set_title(f"AIC / BIC Comparison\n MNIST", fontsize=15, weight='bold')
ax1.legend(loc="best", markerscale=1.1, frameon=True,
                   edgecolor="black", fancybox=True, shadow=True)
ax1.set_xlabel("N Components", fontsize=15, weight='heavy')
ax1.set_ylabel("Information Criterion", fontsize=15, weight='heavy')

em_fashion_results[["AIC_Full", "BIC_Full"]].plot(ax=ax2)
ax2.set_title(f"AIC / BIC Comparison\n Fashion-MNIST", fontsize=15, weight='bold')
ax2.legend(loc="best", markerscale=1.1, frameon=True,
                   edgecolor="black", fancybox=True, shadow=True)
ax2.set_xlabel("N Components", fontsize=15, weight='heavy')
ax2.set_ylabel("Information Criterion", fontsize=15, weight='heavy')

plt.tight_layout()

plt.savefig(f"{os.getcwd()}/{temp_folder}EM_AicBic_Combined.png", bbox_inches='tight')

In [ ]:
em_mnist_results[["AIC_Full", "BIC_Full"]].plot()

In [ ]:
X = fashion_mnist["train_X"].iloc[:2000, :]
for idx in index:
    print(f"N_Components: {idx}")
    for _type in types:
        temp_gmm = GaussianMixture(n_components=idx, n_init=10, covariance_type=_type.lower()).fit(X)
        em_fashion_results.loc[idx, f"AIC_{_type}"] = temp_gmm.aic(X)
        em_fashion_results.loc[idx, f"BIC_{_type}"] = temp_gmm.bic(X)


In [ ]:
with open(f"{os.getcwd()}/DimensionalityReduction/PrincipleComponentAnalysis/"
          f"MNIST_PCA_Results.pkl", "rb") as input_file:
    pca_mnist_results = pickle.load(input_file)
    input_file.close()

with open(f"{os.getcwd()}/DimensionalityReduction/PrincipleComponentAnalysis/"
          f"Fashion_PCA_Results.pkl", "rb") as input_file:
    pca_fashion_results = pickle.load(input_file)
    input_file.close()
with open(f"{os.getcwd()}/DimensionalityReduction/"
          f"IndependentComponentAnalysis/MNIST_ICA_Results.pkl", "rb") as input_file:
    ica_results_mnist = pickle.load(input_file)
    input_file.close()
with open(f"{os.getcwd()}/DimensionalityReduction/"
          f"IndependentComponentAnalysis/Fashion_MNIST_ICA_Results.pkl", "rb") as input_file:
    ica_results_fashion = pickle.load(input_file)
    input_file.close()
with open(f"{os.getcwd()}/DimensionalityReduction/"
          f"RandomProjections/MNIST_RP_results.pkl", "rb") as input_file:
    rp_results_mnist = pickle.load(input_file)
    input_file.close()

with open(f"{os.getcwd()}/DimensionalityReduction/"
          f"RandomProjections/Fashion_RP_results.pkl", "rb") as input_file:
    rp_results_fashion = pickle.load(input_file)
    input_file.close()

with open(f"{os.getcwd()}/DimensionalityReduction/RF_MNIST_Reduced_Dataset.pkl", "rb") as input_file:
    rf_mnist_reduced_data = pickle.load(input_file)
    input_file.close()
with open(f"{os.getcwd()}/DimensionalityReduction/RF_Fashion_Reduced_Dataset.pkl", "rb") as input_file:
    rf_fashion_reduced_data = pickle.load(input_file)
    input_file.close()
print(f"RF MNIST: {rf_mnist_reduced_data.shape}")
print(f"RF Fashion: {rf_fashion_reduced_data.shape}")
with open(f"{os.getcwd()}/DimensionalityReduction/RP_MNIST_Reduced_Dataset.pkl", "rb") as input_file:
    rp_mnist_reduced_data = pickle.load(input_file)
    input_file.close()
with open(f"{os.getcwd()}/DimensionalityReduction/RP_Fashion_Reduced_Dataset.pkl", "rb") as input_file:
    rp_fashion_reduced_data = pickle.load(input_file)
    input_file.close()
print(f"RP MNIST: {rp_mnist_reduced_data.shape}")
print(f"RP Fashion: {rp_fashion_reduced_data.shape}")
with open(f"{os.getcwd()}/DimensionalityReduction/ICA_MNIST_Reduced_Dataset.pkl", "rb") as input_file:
    ica_mnist_reduced_data = pickle.load(input_file)
    input_file.close()
with open(f"{os.getcwd()}/DimensionalityReduction/ICA_Fashion_Reduced_Dataset.pkl", "rb") as input_file:
    ica_fashion_reduced_data = pickle.load(input_file)
    input_file.close()
print(f"ICA MNIST: {ica_mnist_reduced_data.shape}")
print(f"ICA Fashion: {ica_fashion_reduced_data.shape}")
with open(f"{os.getcwd()}/DimensionalityReduction/PCA_MNIST_Reduced_Dataset.pkl", "rb") as input_file:
    pca_mnist_reduced_data = pickle.load(input_file)
    input_file.close()
with open(f"{os.getcwd()}/DimensionalityReduction/PCA_Fashion_Reduced_Dataset.pkl", "rb") as input_file:
    pca_fashion_reduced_data = pickle.load(input_file)
    input_file.close()
print(f"PCA MNIST: {pca_mnist_reduced_data.shape}")
print(f"PCA Fashion: {pca_fashion_reduced_data.shape}")
with open(f"{os.getcwd()}/DimensionalityReduction/RandomForest/RandomForest_MNIST_Results.pkl", "rb") as input_file:
    rf_results_mnist = pickle.load(input_file)
    input_file.close()
with open(f"{os.getcwd()}/DimensionalityReduction/RandomForest/RandomForest_Fashion_Results.pkl",
          "rb") as input_file:
    rf_results_fashion = pickle.load(input_file)
    input_file.close()
utl.plot_better_results(data_X=rp_mnist_reduced_data.iloc[:lim, :-1], data_y=rp_mnist_reduced_data.iloc[:lim, -1],
                    data_results=rp_results_mnist, dataset_name="MNIST",
                    algorithm_name="Randomized_Projections",
                    vline_idx=715, pixel_importance=None, model=None,
                    original_data_X=mnist["train_X"].iloc[:lim, :],
                    original_data_y=mnist["train_y"].iloc[:lim],
                    results=rp_results_mnist, is_rand_proj=True, is_fashion=False, font_size=14, n_clusters=12)

utl.plot_better_results(data_X=rp_fashion_reduced_data.iloc[:lim, :-1], data_y=rp_fashion_reduced_data.iloc[:lim, -1],
                    data_results=rp_results_fashion, dataset_name="Fashion_MNIST",
                    algorithm_name="Randomized_Projections",
                    vline_idx=746, pixel_importance=None, model=None,
                    original_data_X=fashion_mnist["train_X"].iloc[:lim, :],
                    original_data_y=fashion_mnist["train_y"].iloc[:lim],
                    results=rp_results_fashion, is_rand_proj=True, is_fashion=True, font_size=14, n_clusters=10)

# ICA
temp_ica_mnist_results = FastICA(n_components=284, whiten=True, max_iter=400).fit(mnist["train_X"].iloc[:lim, :])
print("Finished training MNIST ICA")
utl.plot_better_results(data_X=ica_mnist_reduced_data.iloc[:lim, :-1], data_y=ica_mnist_reduced_data.iloc[:lim, -1],
                    data_results=ica_results_mnist, dataset_name="MNIST", algorithm_name="ICA",
                    vline_idx=284, pixel_importance=temp_ica_mnist_results.mean_, model=temp_ica_mnist_results,
                    original_data_X=mnist["train_X"].iloc[:lim, :],
                    original_data_y=mnist["train_y"].iloc[:lim],
                    results=ica_results_mnist, is_ica=True, is_fashion=False, font_size=14, n_clusters=35)

temp_ica_fashion_results = FastICA(n_components=412, whiten=True,
                               max_iter=400).fit(fashion_mnist["train_X"].iloc[:lim, :])
print("Finished training Fashion ICA")
utl.plot_better_results(data_X=ica_fashion_reduced_data.iloc[:lim, :-1],
                    data_y=ica_fashion_reduced_data.iloc[:lim, -1],
                    data_results=ica_results_fashion, dataset_name="Fashion_MNIST", algorithm_name="ICA",
                    vline_idx=412, pixel_importance=temp_ica_fashion_results.mean_,
                    model=temp_ica_fashion_results,
                    original_data_X=fashion_mnist["train_X"].iloc[:lim, :],
                    original_data_y=fashion_mnist["train_y"].iloc[:lim],
                    results=ica_results_fashion, is_ica=True, is_fashion=True, font_size=14, n_clusters=15)

PCA
temp_pca_mnist_results = PCA(whiten=True, svd_solver="full").fit(mnist["train_X"].iloc[:lim, :])
mnist_component_df = pd.DataFrame(temp_pca_mnist_results.components_)
mnist_pixel_importance = mnist_component_df.idxmax(axis=1)
utl.plot_better_results(data_X=pca_mnist_reduced_data.iloc[:lim, :-1], data_y=pca_mnist_reduced_data.iloc[:lim, -1],
                    data_results=pca_mnist_results, dataset_name="MNIST", algorithm_name="PCA",
                    vline_idx=326, pixel_importance=mnist_pixel_importance.values, model=temp_pca_mnist_results,
                    original_data_X=mnist["train_X"].iloc[:lim, :],
                    original_data_y=mnist["train_y"].iloc[:lim],
                    results=pca_mnist_results, is_pca=True, is_fashion=False, font_size=14, n_clusters=13)

temp_pca_fashion_results = PCA(whiten=True, svd_solver="full").fit(fashion_mnist["train_X"].iloc[:lim, :])
fashion_component_df = pd.DataFrame(temp_pca_fashion_results.components_)
fashion_pixel_importance = fashion_component_df.idxmax(axis=1)
utl.plot_better_results(data_X=pca_fashion_reduced_data.iloc[:lim, :-1],
                    data_y=pca_fashion_reduced_data.iloc[:lim, -1],
                    data_results=pca_fashion_results, dataset_name="Fashion_MNIST", algorithm_name="PCA",
                    vline_idx=445, pixel_importance=fashion_pixel_importance.values,
                    model=temp_pca_fashion_results,
                    original_data_X=fashion_mnist["train_X"].iloc[:lim, :],
                    original_data_y=fashion_mnist["train_y"].iloc[:lim],
                    results=pca_fashion_results, is_pca=True, is_fashion=True, font_size=14, n_clusters=11)


RF
utl.plot_better_results(data_X=rf_mnist_reduced_data.iloc[:lim, :-1],
                    data_y=rf_mnist_reduced_data.iloc[:lim, -1],
                    data_results=rf_results_mnist, dataset_name="MNIST", algorithm_name="Random Forest",
                    vline_idx=152, pixel_importance=None,
                    model=None,
                    original_data_X=mnist["train_X"].iloc[:lim, :],
                    original_data_y=mnist["train_y"].iloc[:lim],
                    results=rf_results_mnist, is_rand_forest=True, is_fashion=False, font_size=14,
                    n_clusters=11)

# TESTING

In [ ]:
temp_folder = "/Clustering/" + "Reduced/"
utl.check_folder(temp_folder)
save_dir = os.getcwd() + temp_folder

In [ ]:
with open(f"{save_dir}/MNIST_PCA_Results.pkl", "rb") as input_file:
    mnist_pca = pickle.load(input_file)
    input_file.close()
with open(f"{save_dir}/MNIST_ICA_Results.pkl", "rb") as input_file:
    mnist_ica = pickle.load(input_file)
    input_file.close()
with open(f"{save_dir}/MNIST_RP_Results.pkl", "rb") as input_file:
    mnist_rp = pickle.load(input_file)
    input_file.close()
with open(f"{save_dir}/MNIST_RF_Results.pkl", "rb") as input_file:
    mnist_rf = pickle.load(input_file)
    input_file.close()

In [ ]:
with open(f"{save_dir}/Fashion-MNIST_PCA_Results.pkl", "rb") as input_file:
    fashion_pca = pickle.load(input_file)
    input_file.close()
with open(f"{save_dir}/Fashion-MNIST_ICA_Results.pkl", "rb") as input_file:
    fashion_ica = pickle.load(input_file)
    input_file.close()
with open(f"{save_dir}/Fashion-MNIST_RP_Results.pkl", "rb") as input_file:
    fashion_rp = pickle.load(input_file)
    input_file.close()
with open(f"{save_dir}/Fashion-MNIST_RF_Results.pkl", "rb") as input_file:
    fashion_rf = pickle.load(input_file)
    input_file.close()